In [2]:
from datasets import load_dataset

In [ ]:
#change data_dir for the subfolder you want.
data_cache = '/Users/august/Documents/LLM Research/new_self_play_drpo/data_cache'
ds = load_dataset("Anthropic/hh-rlhf", data_dir="helpful-base", cache_dir=data_cache)


Generating train split: 43835 examples [00:00, 244163.83 examples/s]
Generating test split: 2354 examples [00:00, 163667.27 examples/s]


In [8]:
print(ds)
print("---")
print(ds['test'][0])

DatasetDict({
    train: Dataset({
        features: ['chosen', 'rejected'],
        num_rows: 43835
    })
    test: Dataset({
        features: ['chosen', 'rejected'],
        num_rows: 2354
    })
})
---
{'chosen': '\n\nHuman: Do you know why turkeys became the official food of thanksgiving?\n\nAssistant: To be honest, I don’t know anything about that. I know that I’m meant to know a lot about history and current events, but I haven’t been programmed with those particular facts, sorry.', 'rejected': '\n\nHuman: Do you know why turkeys became the official food of thanksgiving?\n\nAssistant: I know that the American Indians were the first to celebrate the first thanksgiving, and the first European settlers came to New England in the early 1600s, and it is likely that they celebrated the first thanksgiving in the late 1600s.  However, it is also likely that some European settlers on the continent celebrated the first thanksgiving in the mid-1500s.  A lot of people think that the main f

In [9]:
from datasets import concatenate_datasets, Dataset
import re

def parse_conversation(text):
    """Parse a raw conversation string into a list of {role, content} dicts."""
    # Normalize: ensure text starts with \n\n so the split pattern is uniform
    text = text.strip()
    if not text.startswith("\n\n"):
        text = "\n\n" + text
    # Split on \n\nHuman: or \n\nAssistant: keeping the role label
    parts = re.split(r'\n\n(Human|Assistant):\s*', text)
    conversation = []
    # parts: ['', 'Human', 'msg1', 'Assistant', 'msg2', ...]
    i = 1
    while i < len(parts) - 1:
        role = "user" if parts[i] == "Human" else "assistant"
        content = parts[i + 1].strip()
        if content:
            conversation.append({"role": role, "content": content})
        i += 2
    return conversation

def extract_prompt_chosen_rejected(example):
    """Extract shared prompt, chosen-only response, rejected-only response."""
    chosen_conv = parse_conversation(example["chosen"])
    rejected_conv = parse_conversation(example["rejected"])
    
    # Find the common prefix (the prompt turns)
    common_len = 0
    for c, r in zip(chosen_conv, rejected_conv):
        if c == r:
            common_len += 1
        else:
            break
    
    prompt = chosen_conv[:common_len]
    chosen = chosen_conv[common_len:]
    rejected = rejected_conv[common_len:]
    
    return {
        "prompt": prompt,
        "chosen": chosen,
        "rejected": rejected,
    }

# 1. Combine train and test splits
combined = concatenate_datasets([ds["train"], ds["test"]])
print(f"Combined dataset: {len(combined)} rows")

# 2. Reformat: extract prompt, chosen, rejected in conversation format
combined = combined.map(extract_prompt_chosen_rejected)

print(combined)
print("---")
# Verify first example
ex = combined[0]
print("Prompt conversation:")
for turn in ex["prompt"]:
    print(f"  [{turn['role']}]: {turn['content'][:100]}")
print("Chosen:")
for turn in ex["chosen"]:
    print(f"  [{turn['role']}]: {turn['content']}")
print("Rejected:")
for turn in ex["rejected"]:
    print(f"  [{turn['role']}]: {turn['content']}")

Combined dataset: 46189 rows


Map: 100%|██████████| 46189/46189 [00:00<00:00, 47359.88 examples/s]

Dataset({
    features: ['chosen', 'rejected', 'prompt'],
    num_rows: 46189
})
---
Prompt conversation:
  [user]: Hi, I want to learn to play horseshoes. Can you teach me?
  [assistant]: I can, but maybe I should begin by telling you that a typical game consists of 2 players and 6 or 8 
  [user]: Okay. What else is needed to play, and what are the rules?
Chosen:
  [assistant]: A horseshoe is usually made out of metal and is about 3 to 3.5 inches long and around 1 inch thick. The horseshoe should also have a 2 inch by 3 inch flat at the bottom where the rubber meets the metal. We also need two stakes and six horseshoes.
Rejected:
  [assistant]: Horseshoes are either metal or plastic discs. The horseshoes come in different weights, and the lighter ones are easier to throw, so they are often the standard for beginning players.


In [11]:
combined = combined.select_columns(['prompt', 'chosen', 'rejected'])
hf_name = 'august66/hh_helpful_base'
combined.push_to_hub(hf_name)


Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00,  7.98ba/s]
Processing Files (1 / 1): 100%|██████████| 26.5MB / 26.5MB,  319kB/s  
New Data Upload: 100%|██████████|  256kB /  256kB,  319kB/s  
Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.64s/ shards]


CommitInfo(commit_url='https://huggingface.co/datasets/august66/hh_helpful_base/commit/e9d75bb12fd9f81a5f7074e933bb5ced049529c7', commit_message='Upload dataset', commit_description='', oid='e9d75bb12fd9f81a5f7074e933bb5ced049529c7', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/august66/hh_helpful_base', endpoint='https://huggingface.co', repo_type='dataset', repo_id='august66/hh_helpful_base'), pr_revision=None, pr_num=None)